## Ensure a Minikube cluster is running

- If you created a Minikube cluster with the command `minikube start --nodes 3` and it is still running:
  - Do nothing
- If you have a stopped (not deleted) Minikube cluster that was created with the command `minikube start --nodes 3`:
  - Run the command `minikube start`
- If you don't have a Minikube cluster (or deleted your previous one):
  - Run the command `minikube start --nodes 3`  

## Deploy a Pod

- The Pod is defined in the YAML file `manifests/myapp.yaml`

In [1]:
!kubectl apply -f manifests/myapp.yaml

pod/myapp-pod created


## Let's look at the Pod's YAML

**Note:**

- The Pod's container is listening on `containerPort` 80.
- The Pod's `labels` are `app: myapp` and `type: front-end`.

```bash
apiVersion: v1
kind: Pod            # we are defining a Pod
metadata:
  name: myapp-pod    # the Pod's name is myapp-pod
  labels:
    app: myapp       # the Pod has two labels defined:
    type: front-end  # app = myapp and type = front-end
spec:
  containers:
  - name: nginx-container  # the name of the container (running in the Pod)
    image: nginx           # the image the container is based on
    resources:
      requests:            # minimum requested CPU/RAM for the container
        cpu: 100m
        memory: 128Mi
      limits:              # maximum allowable CPU/RAM for the container
        cpu: 250m
        memory: 256Mi    
    ports:
    - containerPort: 80    # the container is listening on port 80
```

In [2]:
#!type manifests/myapp.yaml # use this on Windows
!cat manifests/myapp.yaml

apiVersion: v1
kind: Pod
metadata:
  name: myapp-pod
  labels:
    app: myapp # change this to: myapp2
    type: front-end
spec:
  containers:
  - name: nginx-container
    image: nginx
    resources:
      requests:
        cpu: 100m
        memory: 128Mi
      limits:
        cpu: 250m
        memory: 256Mi    
    ports:
    - containerPort: 80

## Deploy a Service

- The Service is defined in the YAML file `manifests/myservice.yaml`

In [3]:
!kubectl apply -f manifests/myservice.yaml

service/myservice created


## Let's look at the Pod's YAML

**Note:**

- The Service is listening on `port` 80 (on its own IP address) and redirects traffic to `targetPort` 80 (on another IP address).
- The Service's `targetPort` matches the Pod's `containerPort`.
  - The Service will output traffic to port 80 (`targetPort`).
  - The Pod's container is listening on port 80 (`containerPort`).
- The Service's `selector` labels match the Pod's `labels` (`app: myapp` and `type: front-end`)
  - Therefore, the Service will redirect traffic to the Pod (and its container).
  - If more Pod's had the same `labels`, the Service would (round robin) load balance between all matching Pods.

```bash
apiVersion: v1
kind: Service         # we are defining a Service
metadata:
  name: myservice     # the Service's name is myservice
spec:
  ports:
  - port: 80          # the service is listening on port 80
    targetPort: 80    # the service is redirecting traffic to port 80
  selector:
    app: myapp        # the Service is selecting Pods that have two labels defined:
    type: front-end   # app = myapp and type = front-end
```

In [4]:
#!type manifests/myservice.yaml # use this on Windows
!cat manifests/myservice.yaml

apiVersion: v1
kind: Service
metadata:
  name: myservice
spec:
  ports:
  - port: 80
    targetPort: 80
  selector:
    app: myapp
    type: front-end

## List Pods

- Notice the `IP` address of the pod.

In [5]:
!kubectl get po -o wide
#!kubectl get pods -o wide

NAME        READY   STATUS    RESTARTS   AGE   IP            NODE           NOMINATED NODE   READINESS GATES
myapp-pod   1/1     Running   0          13s   10.244.1.16   minikube-m02   <none>           <none>


## List Pod's container name and containerPort

- Notice the `containerPort` the container is listening on.

In [6]:
!kubectl get pod myapp-pod -o jsonpath='{range .spec.containers[*]}name: {.name}, containerPort: {.ports[*].containerPort}{"\n"}{end}'

name: nginx-container, containerPort: 80


## List Service Endpoints (ep)

- If the Service is connected to the Pod, the Service Endpoint will point to the Pod's IP address.
- Notice the Service Endpoint (`ENDPOINTS`)
  - Matches the Pod IP address (`IP`) above.
  - Matches the container's port (`containerPort`) above.
- If the Service was load balancing between multiple Pods, there would be more IP addresses listed under `ENDPOINTS`.

In [7]:
#!kubectl get ep myservice
!kubectl get endpoints myservice

NAME        ENDPOINTS        AGE
myservice   10.244.1.16:80   17s


## List Services

- Our Service is called `myservice`.
- Under `PORT(S)` we see it is listening on port 80.

In [8]:
#!kubectl get svc -o wide
!kubectl get services -o wide

NAME         TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE     SELECTOR
kubernetes   ClusterIP   10.96.0.1       <none>        443/TCP   6h42m   <none>
myservice    ClusterIP   10.105.34.114   <none>        80/TCP    21s     app=myapp,type=front-end


## Port forward your host machine's port 8080 to the service's port 80

- Traffic Flow:  Hostmachine (8080) <--> Service (80) <--> Pod's Container (80).
- The container is running an nginx web server on port 80 (that serves a default web page).

### Run these commands in a terminal (won't work from a notebook cell)

```bash
# Port forward to the service
kubectl port-forward service/myservice 8080:80

# --- Output ---
# Forwarding from 127.0.0.1:8080 -> 80
# Forwarding from [::1]:8080 -> 80
# --------------

# Open a browser and enter the URL http://localhost:8080
# or open a new terminal (on Ubuntu/masOS) an run the command below:
curl http://localhost:8080

# --- Output ---
# <!DOCTYPE html>
# <html>
# <head>
# <title>Welcome to nginx!</title>
# <style>
# html { color-scheme: light dark; }
# body { width: 35em; margin: 0 auto;
# font-family: Tahoma, Verdana, Arial, sans-serif; }
# </style>
# </head>
# <body>
# <h1>Welcome to nginx!</h1>
# <p>If you see this page, the nginx web server is successfully installed and
# working. Further configuration is required.</p>
# 
# <p>For online documentation and support please refer to
# <a href="http://nginx.org/">nginx.org</a>.<br/>
# Commercial support is available at
# <a href="http://nginx.com/">nginx.com</a>.</p>
# 
# <p><em>Thank you for using nginx.</em></p>
# </body>
# </html>
# --------------

# Stop the port forward by typing Ctrl + C in the terminal

# --- Output ---
# ^C
# --------------
```

## Edit the Pod's YAML file

- Open the Pod's YAML file `manifests/myapp.yaml`.
- Change the `app` label to `myapp2`.
- Save the file.

```bash
apiVersion: v1
kind: Pod
metadata:
  name: myapp-pod
  labels:
    app: myapp2 # change this to: myapp2
    type: front-end
spec:
  containers:
  - name: nginx-container
    image: nginx
    resources:
      requests:
        cpu: 100m
        memory: 128Mi
      limits:
        cpu: 250m
        memory: 256Mi    
    ports:
    - containerPort: 80
```

## Deploy (apply) the Pod's modified YAML file

In [9]:
!kubectl apply -f manifests/myapp.yaml

pod/myapp-pod configured


## Check the Service Endpoint again

- Notice the Service's `ENDPOINTS` is empty (`<none>`).
  - The service's `selector` can't find any Pods with the labels `app: myapp` and `type: front-end`.
  - This is because the Pod's labels are now  `app: myapp2` and `type: front-end`.

In [10]:
#!kubectl get ep myservice
!kubectl get endpoints myservice

NAME        ENDPOINTS   AGE
myservice   <none>      78s


## Try to port forward to the Service again

### Run these commands in a terminal (won't work from a notebook cell)

```bash
## Port forward port 8080 to the service (that's listening on port 80)
kubectl port-forward service/myservice 8080:80

# --- Output ---
# error: timed out waiting for the condition
# --------------

# If not stopped, stop the port forward by typing **Ctrl-C** in the terminal

# --- Output ---
# ^C
# --------------
```

## Edit the Pod's YAML file

- Open the Pod's YAML file `manifests/myapp.yaml`.
- Change the `app` label back to `myapp`.
- Save the file.

```bash
apiVersion: v1
kind: Pod
metadata:
  name: myapp-pod
  labels:
    app: myapp # change this to: myapp
    type: front-end
spec:
  containers:
  - name: nginx-container
    image: nginx
    resources:
      requests:
        cpu: 100m
        memory: 128Mi
      limits:
        cpu: 250m
        memory: 256Mi    
    ports:
    - containerPort: 80
```

## Deploy (apply) the Pod's modified YAML file

In [11]:
!kubectl apply -f manifests/myapp.yaml

pod/myapp-pod configured


## Check the Service Endpoint again

- Notice the Service's `ENDPOINTS` once again contains an IP address.
  - The service's `selector` once again finds a Pod with the labels `app: myapp` and `type: front-end`.
  - This is because the Pod's labels are now  `app: myapp` and `type: front-end`.

In [12]:
#!kubectl get ep myservice
!kubectl get endpoints myservice

NAME        ENDPOINTS        AGE
myservice   10.244.1.16:80   111s


## Delete the Service and Pod

In [13]:
!kubectl delete -f manifests/myservice.yaml
!kubectl delete -f manifests/myapp.yaml

service "myservice" deleted
pod "myapp-pod" deleted


## List Services and Pods

- We see that the Service and Pods have been deleted.

In [14]:
#!kubectl get svc
#!kubectl get po
!kubectl get services
!kubectl get pods

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   6h44m
No resources found in default namespace.
